# String Resemblance Grouping

## Table of Contents

* [Introduction](#introduction)
* [Windows Log Example](#windows-logs-example)
    * [Dataset](#dataset)
    * [Data Processing and Sampling](#data-processing-and-sampling)
    * [Model Training](#model-training)
    * [Results](#results)
        * [Group Counts](#group-counts)
        * [Number of unique logs trained over](#number-of-unique-logs-trained-over)
        * [Number of groups identified by model training](#number-of-groups-identifed-by-model-training)
        * [Exploring group spread](#exploring-group-spread)
* [Conclusions](#conclusions)
* [References](#references)

## Introduction
SRG is designed to find a subset of representative strings within a a large collection of messages. These representative strings create groupings with which to categorize the messages for further exploration or triage.

## Windows Logs Example

### Dataset
Some example logs are available through Zenodo. This notebook will be utilizing Windows logs.


In [1]:
import urllib.request
url = 'https://zenodo.org/record/3227177/files/Windows.tar.gz'
filename = '../datasets/Windows.tar.gz'
urllib.request.urlretrieve(url, filename)

('../datasets/Windows.tar.gz', <http.client.HTTPMessage at 0x7fbcc43e8b50>)

In [2]:
from subprocess import call
call(['tar', '-xzf', '../datasets/Windows.tar.gz', '-C', '../datasets/'])

0

In [3]:
import os
import sys
import glob
srg_path = os.path.abspath('../')
if srg_path not in sys.path:
    sys.path.append(srg_path)
from srg import SRG
import dask_cudf

In [4]:
import time

from dask.distributed import Client, wait
from dask_cuda import LocalCUDACluster

cluster = LocalCUDACluster()
client = Client(cluster)
client

/home/nfs/sdavis/miniconda3/envs/srg/lib/python3.8/site-packages/distributed/node.py:180: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 46491 instead
  warnings.warn(
2022-10-28 16:30:31,977 - distributed.diskutils - INFO - Found stale lock file and directory '/home/nfs/sdavis/morpheus-experimental/syslog-resemblance-grouping/training-tuning/dask-worker-space/worker-guvrkzph', purging
2022-10-28 16:30:31,978 - distributed.diskutils - INFO - Found stale lock file and directory '/home/nfs/sdavis/morpheus-experimental/syslog-resemblance-grouping/training-tuning/dask-worker-space/worker-e5aet1o0', purging
2022-10-28 16:30:31,980 - distributed.diskutils - INFO - Found stale lock file and directory '/home/nfs/sdavis/morpheus-experimental/syslog-resemblance-grouping/training-tuning/dask-worker-space/worker-qq28u2o7', purging
2022-10-28 16:30:31,981 - distributed.diskutils - INFO - Found stale lock file and directory '/hom

Connection method: Cluster object,Cluster type: dask_cuda.LocalCUDACluster
Dashboard: http://127.0.0.1:46491/status,
Dashboard: http://127.0.0.1:46491/status,Workers: 8
Total threads: 8,Total memory: 503.79 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:34969,Workers: 8
Dashboard: http://127.0.0.1:46491/status,Total threads: 8
Started: Just now,Total memory: 503.79 GiB
Comm: tcp://127.0.0.1:39947,Total threads: 1
Dashboard: http://127.0.0.1:41937/status,Memory: 62.97 GiB
Nanny: tcp://127.0.0.1:42143,


### Data processing and sampling
The logs are loaded into a Dask cudf object, excess whitespace is collapsed into a single space, and then a sample of the logs is taken for training.

In [5]:
win_log_dir = '../datasets/Windows.log'
win_logs = dask_cudf.read_csv(win_log_dir, delimiter = ',', names = ['ts', 'log']).dropna()

In [6]:
def collapse_whitespace(x):
    return ' '.join(x.split())

In [7]:
def clean_up(df):
    pdf = df.to_pandas().copy()
    pdf['cleaned'] = pdf.apply(lambda row: collapse_whitespace(row['log']), axis=1)
    return pdf

In [8]:
L = win_logs.sample(frac=0.001).map_partitions(lambda df: clean_up(df))

In [9]:
len(L)

114535

### Model Training

In [10]:
win_srg = SRG()
win_srg.fit(L, column='cleaned', shingle_size=4)

Current dimension:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/5 [00:00<?, ?it/s]

### Results

In [11]:
win_labeled = win_srg.transform(L, column='cleaned')

#### Group counts

In [12]:
label_counts = win_labeled[['srg_label', 'srg_rep']].groupby('srg_rep').count().compute()
label_counts

,srg_label
srg_rep,
Info CBS Appl: DetectUpdate,180
Info CBS Appl: Evaluating package applicability for package Package_30_for_KB2791765~31bf3856ad364e35~amd64~~6.1.1.2,2128
Info CBS Appl: Partial install Status testing,70
Info CBS Appl: SelfUpdate detect,45461
Info CBS Appl: detect Parent,8803
Info CBS Appl: detectParent: package: Microsoft-Windows-RDP-BlueIP-Package-TopLevel~31bf3856ad364e35~amd64~~7.2.7601.16415,29
Info CBS Appl: detectParent: package: Package_60_for_KB3147071~31bf3856ad364e35~amd64~~6.1.1.1,8728
Info CBS Applicability(ComponentAnalyzerEvaluateSelfUpdate): Component: wow64_microsoft-windows-p..gssystems.resources_31bf3856ad364e35_6.1.7601.22183_pl-pl_c4015429ab7cd276,2284
Info CBS Applicability(ComponentAnalyzerEvaluateSelfUpdate): Component: wow64_microsoft-windows-systemrestore-main_31bf3856ad364e35_6.1.7601.23136_none_afd04622176b963e,414


#### Number of unique logs trained over

In [13]:
len(win_labeled['cleaned'].unique())

15610

#### Number of groups identifed by model training

In [14]:
len(win_srg._groups)

45

#### Exploring group spread
The below calculates the resemblance of each log to it's representative and finds the maximum, average, and standard deviation of these resemblances for each group. This provides insight into how well each group is represented. This could also inform additional subsetting and further SRG training if there's a large, spread out group that warrants further investigation.

In [15]:
from srg.fastmap.distances import Jaccard

In [16]:
def rep_dists(x, y):
    return Jaccard(shingle_size=4).calculate(x, y)

In [17]:
def calc_dists(df):
    pdf = df.copy()
    pdf['rep_dist'] = pdf.apply(lambda row: rep_dists(row['cleaned'], row['srg_rep']), axis=1)
    return pdf

In [18]:
labeled_dists = win_labeled.map_partitions(lambda df: calc_dists(df))

In [19]:
avg_rep_dists = labeled_dists[['srg_label', 'srg_rep', 'rep_dist']].groupby(['srg_label', 'srg_rep']).agg({'rep_dist': ['count', 'max', 'mean', 'std']}).compute()

In [20]:
avg_rep_dists

rep_dist  \
                                                                count   
srg_label srg_rep                                                       
0         Info CBS Appl: SelfUpdate detect                      45461   
1         Info CBS Appl: DetectUpdate                             180   
2         Info CBS Appl: detect Parent                           8803   
3         Info CBS Appl: Partial install Status testing            70   
4         Info CBS EvaluateApplicability                         1684   
5         Info CSI 00001083 Transaction merge required             35   
6         pA = PROCESSOR_ARCHITECTURE_INTEL (0)                    19   
7         Version = 6.1.7601.23714                                 59   
8         Version = 6.1.7600.16385                                 25   
9         Version = 11.2.9600.18059                                18   
10        Info DPX Started DPX phase: Apply Deltas Provid...        3   
11        Info CBS Startup: Changing logon timeout to a s...     2407   
12        Info CSI geComponent (13): flags: 0 app: (null)...        4   
13        Info CBS Appl: detectParent: package: Package_6...     8728   
14        Info CBS Plan: Package: Package_128_for_KB29733...      235   
15        Info CBS Appl: Package: Microsoft-Windows-IIS-W...        5   
16        Info CBS Appl: detectParent: package: Microsoft...        4   
17        Info CBS Appl: detectParent: package: Microsoft...       29   
18        Info CBS Appl: detectParent: package: Microsoft...        3   
19        Info CBS Exec: Resolving Package: Microsoft-Win...        5   
20        Info CBS Exec: Installing Package: Microsoft-Wi...        5   
21        Info CBS Exec: Resolving Package: Microsoft-Win...        5   
22        Info CBS Appl: Evaluating package applicability...     2128   
23        Info CBS Plan: Merge into existing execution pa...        9   
24        Info CSI 00001894 ICSITransaction::Commit calli...        2   
25        Info CSI 00000024 Begin executing advanced inst...        2   
26        Info CBS Unloading offline registry hive: {bf1a...        3   
28        Info CBS Exec: Staging Package: Microsoft-Windo...        1   
29        Info CBS Startup: Package: Microsoft-Windows-WM...       31   
30        Info CBS Applicability(ComponentAnalyzerEvaluat...      414   
31        Info CBS Applicability(ComponentAnalyzerEvaluat...     2284   
32        Info CBS Applicability(ComponentAnalyzerEvaluat...    40709   
33        Info CBS Applicability(ComponentAnalyzerEvaluat...      617   
34        Info CBS Failed to get session package state fo...      157   
35        Info CBS Applicability(ComponentAnalyzerEvaluat...      124   
36        Info CBS Applicability(ComponentAnalyzerEvaluat...       13   
38        Info CBS Plan: Skipping package since its start...        3   
40        Info CBS Exec: Reboot required for Package: Pac...        2   
41        Info CSI 00000bde@2016/11/21:19:15:20.293 CSI T...       22   
42        Info CSI 000000ae Processing Namespace [wcm://M...        3   
43        Info CBS WatchList: Add package Package_130_for...      218   
44        Info CSI 0000003d Processing Namespace uninstal...        3   
27        Info CBS Unloading offline registry hive: {bf1a...        1   
37        l:324{162}]"Microsoft-Windows-IE-InternetExplor...        1   
39        Info CBS Exec: Reboot required for Package: Pac...        1   

                                                                        \
                                                                   max   
srg_label srg_rep                                                        
0         Info CBS Appl: SelfUpdate detect                    0.454545   
1         Info CBS Appl: DetectUpdate                         0.000000   
2         Info CBS Appl: detect Parent                        0.000000   
3         Info CBS Appl: Partial install Status testing       0.000000   
4         Info CBS EvaluateApplicabili

### Conclusions
This shows how logs (or any string data) can be syntactically grouped with minimal *a priori* parameterization. After fitting the model to your data, group statistics can be calculated for dataset insights or specific groups of interest can be further explored for triage and informed filtering. This filtering and triage could be particularly helpful in incident investigations or root cause analyses. Models can be saved for future inferencing.

### References
https://github.com/logpai/loghub